<a href="https://colab.research.google.com/github/neversettlejay/mineai_cote_gold/blob/initialization_and_preprocessing/MineAI_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
'''MineAI - Open Pit Blast Rating and Optimization'''
#
# This project focuses on using algorithms and available data to rate the effectiveness of open pit blasts.
# The goal is to suggest optimal parameters that can improve drill and blast efficiency, as well as reduce the
# deviation between planned and actual drilling results (e.g., drill hole deviations, blast outcomes, etc.).
# Through data analysis and predictive modeling, the project aims to enhance mining operations by optimizing blast
# designs, improving fragmentation, and reducing unplanned costs due to inefficiencies in drilling and blasting.
#
# Team Members:
# 1) Manav Chaudhary - Mining Engineer
# 2) Shrey Patel - Machine Learning Engineer
# 3) Soham Salakhana - Mining Engineer
# 4) Shakil Kalvatar - Mining Engineer
# 5) Jay Rathod - Software Engineer
#
# The project will leverage:
# - Historical data from past drill and blast operations
# - Sensor data from the mine, including real-time measurements of blast outcomes and drill hole characteristics
# - Statistical methods, machine learning algorithms, and optimization techniques to rate blast performance
# - Suggestions for optimizing parameters like drill depth, spacing, and blast charge amounts
#
# Key objectives:
# - Rate the success of each blast based on fragmentation size, efficiency, and deviation from the planned design.
# - Use machine learning algorithms to identify patterns in the data and suggest improvements to drill and blast parameters.
# - Propose changes to the drilling process to reduce deviation and optimize efficiency, ultimately improving cost and time savings.


'MineAI - Open Pit Blast Rating and Optimization'

In [7]:
#importing the given data from google drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the file in Google Drive
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the parent directory path
directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'

# List of the subfolders to include (if only these are relevant)
included_folders = ['Data1of3', 'Data2of3', 'Data3of3']

# List to store CSV file paths
csv_files = []

# Walk through the directory structure
for root, dirs, files in os.walk(directory_path):
    # Extract the folder name from the root path
    folder_name = root.split(os.sep)[-2]  # Get the immediate parent folder name (if we would have chosen -3 then it would have given us the parent directory of the parent directory)
    # Only process the folder if it is one of the included folders
    if folder_name in included_folders:
        for file in files:
            # Check if the file is a CSV file
            if file.endswith('.csv'):
                # Get the full path of the file
                full_file_path = os.path.join(root, file)
                csv_files.append(full_file_path)

# Display all the CSV files found
for csv_file in csv_files:
    print(csv_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_137/C1_352_137.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_121/C1_352_121.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_132/C1_352_132.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_131/C1_352_131.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_352_108/C1_352_108.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_352_103/C1_352_103.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_328_109/C1_328_109.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_340_102/C1_340_102.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_364_303/C1_364_303.csv
/content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_352_312/C1_352_312.csv
/content/drive/My Drive/pour_la

In [19]:
import os
import pandas as pd
import plotly.graph_objects as go

# Function to plot drill holes from a given CSV file
def plot_drill_holes_from_csv(csv_file_path):
    """
    This function reads a CSV file containing drill hole data, extracts necessary columns,
    and plots a 3D scatter plot representing the drill holes.

    Args:
    csv_file_path (str): The file path of the CSV file containing the drill hole data.

    Returns:
    None: Displays the 3D plot of drill holes.
    """

    # Step 1: Read the CSV data into a pandas DataFrame
    try:
        df = pd.read_csv(csv_file_path)
        print(f"Successfully loaded data from: {csv_file_path}")
    except Exception as e:
        print(f"Error loading CSV file: {csv_file_path}. Error: {e}")
        return

    # Step 2: Extract necessary columns for the plot
    required_columns = ['Drillhole.X', 'Drillhole.Y', 'Drillhole.Z', 'Drillhole.ToeX', 'Drillhole.ToeY', 'Drillhole.ToeZ']

    # Check if all required columns are present in the dataframe
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Missing columns in {csv_file_path}: {', '.join(missing_columns)}")
        return

    # Extract coordinates
    start_X = df['Drillhole.X'].tolist()
    start_Y = df['Drillhole.Y'].tolist()
    start_Z = df['Drillhole.Z'].tolist()
    toe_X = df['Drillhole.ToeX'].tolist()
    toe_Y = df['Drillhole.ToeY'].tolist()
    toe_Z = df['Drillhole.ToeZ'].tolist()

    # Step 3: Create a 3D plot for drill holes
    fig = go.Figure()

    # Loop through each drill hole and add a trace to the plot
    for i in range(len(start_X)):
        fig.add_trace(go.Scatter3d(
            x=[start_X[i], toe_X[i]],
            y=[start_Y[i], toe_Y[i]],
            z=[start_Z[i], toe_Z[i]],
            mode='lines+markers',
            line=dict(color='blue', width=2),
            marker=dict(size=6, color='red'),
            name=f"Drill Hole {i+1}"
        ))

    # Step 4: Customize the layout of the plot for better visualization
    fig.update_layout(
        scene=dict(
            xaxis_title='X Coordinate (m)',
            yaxis_title='Y Coordinate (m)',
            zaxis_title='Z Coordinate (m)',
            xaxis=dict(showgrid=True, zeroline=True),
            yaxis=dict(showgrid=True, zeroline=True),
            zaxis=dict(showgrid=True, zeroline=True)
        ),
        title=f"Interactive 3D View of Drill Holes\nData from {csv_file_path}",
        title_font=dict(size=16),
        autosize=True,
        height=600
    )

    # Step 5: Display the 3D plot
    fig.show()

# Function to search for and plot CSV files from specified folders
def plot_drill_holes_in_directory(directory_path, included_folders):
    """
    This function walks through a directory and its subdirectories, finds all CSV files
    in the specified folders, and generates 3D plots for each CSV file found.

    Args:
    directory_path (str): The root path to search for the folders containing drill hole data.
    included_folders (list): List of folder names to include in the search (e.g., 'Data1of3', 'Data2of3').

    Returns:
    None: Displays 3D plots for all the found CSV files.
    """

    # Step 1: Loop through all subdirectories of the given directory
    for root, dirs, files in os.walk(directory_path):
        # Extract the immediate parent folder name
        folder_name = root.split(os.sep)[-2]

        # Step 2: Only consider the specified folders
        if folder_name in included_folders:
            print(f"\nFound folder: {folder_name} - Searching for CSV files...")

            # Step 3: Loop through the files and find CSV files
            for file in files:
                if file.endswith('.csv'):
                    csv_file_path = os.path.join(root, file)
                    print(f"Found CSV file: {csv_file_path}")

                    # Step 4: Plot the drill holes for the current CSV file
                    plot_drill_holes_from_csv(csv_file_path)

# Directory path where the data is stored (your Google Drive path)
directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'

# Folders to consider for CSV files (Data1of3, Data2of3, Data3of3)
included_folders = ['Data1of3', 'Data2of3', 'Data3of3']

# Step 5: Call the function to search for CSV files and plot them
plot_drill_holes_in_directory(directory_path, included_folders)



Found folder: Data2of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_137/C1_352_137.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_137/C1_352_137.csv



Found folder: Data2of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_121/C1_352_121.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_121/C1_352_121.csv



Found folder: Data2of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_132/C1_352_132.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_132/C1_352_132.csv



Found folder: Data2of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_131/C1_352_131.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data2of3/C1_352_131/C1_352_131.csv



Found folder: Data1of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_352_108/C1_352_108.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_352_108/C1_352_108.csv


...


Found folder: Data1of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_340_102/C1_340_102.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data1of3/C1_340_102/C1_340_102.csv



Found folder: Data3of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_364_303/C1_364_303.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_364_303/C1_364_303.csv



Found folder: Data3of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_352_312/C1_352_312.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_352_312/C1_352_312.csv



Found folder: Data3of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_364_105/C1_364_105.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_364_105/C1_364_105.csv



Found folder: Data3of3 - Searching for CSV files...
Found CSV file: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_352_138/C1_352-138.csv
Successfully loaded data from: /content/drive/My Drive/pour_la_hackathon/MineAI/Data3of3/C1_352_138/C1_352-138.csv
